In [1]:
!python -V

Python 3.9.19


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error, mean_squared_error

In [5]:
!mlflow --version

mlflow, version 2.13.0


In [6]:
import mlflow

!mlflow --version

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

mlflow, version 2.13.0


<Experiment: artifact_location=('file:///c:/Users/ramsi/OneDrive/Documents/Github '
 'Projects/mlops-zoomcamp/02-experiment-tracking/mlruns/1'), creation_time=1716844413883, experiment_id='1', last_update_time=1716844413883, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [7]:
def read_dataframe(filename):
    if filename.endswith(".csv"):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith(".parquet"):
        df = pd.read_parquet(filename)

    df["duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ["PULocationID", "DOLocationID"]
    df[categorical] = df[categorical].astype(str)

    return df

In [8]:
df_train = read_dataframe("data/green_tripdata_2021-01.parquet")
df_val = read_dataframe("data/green_tripdata_2021-02.parquet")

In [9]:
len(df_train), len(df_val)

(73908, 61921)

In [10]:
df_train["PU_DO"] = df_train["PULocationID"] + "_" + df_train["DOLocationID"]
df_val["PU_DO"] = df_val["PULocationID"] + "_" + df_val["DOLocationID"]

In [11]:
categorical = ["PU_DO"]  #'PULocationID', 'DOLocationID']
numerical = ["trip_distance"]

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient="records")
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient="records")
X_val = dv.transform(val_dicts)

In [14]:
target = "duration"
y_train = df_train[target].values
y_val = df_val[target].values

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\metrics\_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


7.758715205396466

In [25]:
with open("models/lin_reg.bin", "wb") as f_out:
    pickle.dump((dv, lr), f_out)

In [31]:
lr = Lasso(0.5)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

root_mean_squared_error(y_val, y_pred)

12.212582619743838

with mlflow.start_run():

    mlflow.set_tag("developer", "ramsi")

    mlflow.log_param("train-data-path", "data/green_tripdata_2021-01.parquet")
    mlflow.log_param("valid-data-path", "data/green_tripdata_2021-02.parquet")

    alpha = 0.1
    mlflow.log_param("alpha", alpha)
    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    mlflow.log_artifact(
        local_path="models/lin_reg.bin", artifact_path="models_pickle"
    )


In [12]:
import xgboost as xgb

In [35]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [15]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [16]:
def objective(params):
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, "validation")],
            early_stopping_rounds=50,
        )
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)
        mlflow.log_metric("rmse", rmse)

    return {"loss": rmse, "status": STATUS_OK}

In [41]:
search_space = {
    "max_depth": scope.int(hp.quniform("max_depth", 4, 100, 1)),
    "learning_rate": hp.loguniform("learning_rate", -3, 0),
    "reg_alpha": hp.loguniform("reg_alpha", -5, -1),
    "reg_lambda": hp.loguniform("reg_lambda", -6, -1),
    "min_child_weight": hp.loguniform("min_child_weight", -1, 3),
    "objective": "reg:linear",
    "seed": 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials(),
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [09:39:27] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.77803                          
[1]	validation-rmse:11.37241                          
[2]	validation-rmse:10.99503                          
[3]	validation-rmse:10.64400                          
[4]	validation-rmse:10.31805                          
[5]	validation-rmse:10.01553                          
[6]	validation-rmse:9.73475                           
[7]	validation-rmse:9.47540                           
[8]	validation-rmse:9.23575                           
[9]	validation-rmse:9.01420                           
[10]	validation-rmse:8.80977                          
[11]	validation-rmse:8.62117                          
[12]	validation-rmse:8.44767                          
[13]	validation-rmse:8.28808                          
[14]	validation-rmse:8.14092                          
[15]	validation-rmse:8.00568                          
[16]	validation-rmse:7.88118                          
[17]	validation-rmse:7.76754                          
[18]	valid

c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [09:42:35] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:10.63398                                                     
[1]	validation-rmse:9.46973                                                      
[2]	validation-rmse:8.62516                                                      
[3]	validation-rmse:8.02336                                                      
[4]	validation-rmse:7.59979                                                      
[5]	validation-rmse:7.30728                                                      
[6]	validation-rmse:7.09685                                                      
[7]	validation-rmse:6.95138                                                      
[8]	validation-rmse:6.84828                                                      
[9]	validation-rmse:6.77554                                                      
[10]	validation-rmse:6.71964                                                     
[11]	validation-rmse:6.67919                                                     
[12]	validation-

...

[0]	validation-rmse:11.51468                                                    
[1]	validation-rmse:10.89375                                                    
[2]	validation-rmse:10.34343                                                    
[3]	validation-rmse:9.85644                                                     
[4]	validation-rmse:9.42782                                                     
[5]	validation-rmse:9.04893                                                     
[6]	validation-rmse:8.71882                                                     
[7]	validation-rmse:8.42921                                                     
[8]	validation-rmse:8.17683                                                     
[9]	validation-rmse:7.95769                                                     
[10]	validation-rmse:7.76688                                                    
[11]	validation-rmse:7.60200                                                    
[12]	validation-rmse:7.45832

c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [10:32:22] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.72845                                                    
[1]	validation-rmse:11.28112                                                    
[2]	validation-rmse:10.86888                                                    
[3]	validation-rmse:10.48941                                                    
[4]	validation-rmse:10.14066                                                    
[5]	validation-rmse:9.82044                                                     
[6]	validation-rmse:9.52760                                                     
[7]	validation-rmse:9.25924                                                     
[8]	validation-rmse:9.01342                                                     
[9]	validation-rmse:8.78900                                                     
[10]	validation-rmse:8.58441                                                    
[11]	validation-rmse:8.39804                                                    
[12]	validation-rmse:8.22837

c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [10:35:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)



[0]	validation-rmse:11.53996                                                     
[1]	validation-rmse:10.94112                                                     
[2]	validation-rmse:10.40983                                                     
[3]	validation-rmse:9.93943                                                      
[4]	validation-rmse:9.52484                                                      
[5]	validation-rmse:9.16015                                                      
[6]	validation-rmse:8.84037                                                      
[7]	validation-rmse:8.56063                                                      
[8]	validation-rmse:8.31642                                                      
[9]	validation-rmse:8.10385                                                      
[10]	validation-rmse:7.91910                                                     
[11]	validation-rmse:7.75830                                                     
[12]	validation-

In [42]:
mlflow.xgboost.autolog(disable=True)

In [43]:
with mlflow.start_run():

    train = xgb.DMatrix(X_train, label=y_train)
    valid = xgb.DMatrix(X_val, label=y_val)

    best_params = {
        "learning_rate": 0.09585355369315604,
        "max_depth": 30,
        "min_child_weight": 1.060597050922164,
        "objective": "reg:linear",
        "reg_alpha": 0.018060244040060163,
        "reg_lambda": 0.011658731377413597,
        "seed": 42,
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, "validation")],
        early_stopping_rounds=50,
    )

    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    with open("models/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)
    mlflow.log_artifact("models/preprocessor.b", artifact_path="preprocessor")

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [10:38:20] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\objective\regression_obj.cu:209: reg:linear is now deprecated in favor of reg:squarederror.
  warnings.warn(smsg, UserWarning)


[0]	validation-rmse:11.44482
[1]	validation-rmse:10.77202
[2]	validation-rmse:10.18363
[3]	validation-rmse:9.67396
[4]	validation-rmse:9.23166
[5]	validation-rmse:8.84808
[6]	validation-rmse:8.51883
[7]	validation-rmse:8.23597
[8]	validation-rmse:7.99320
[9]	validation-rmse:7.78709
[10]	validation-rmse:7.61022
[11]	validation-rmse:7.45952
[12]	validation-rmse:7.33049
[13]	validation-rmse:7.22098
[14]	validation-rmse:7.12713
[15]	validation-rmse:7.04752
[16]	validation-rmse:6.98005
[17]	validation-rmse:6.92232
[18]	validation-rmse:6.87112
[19]	validation-rmse:6.82740
[20]	validation-rmse:6.78995
[21]	validation-rmse:6.75792
[22]	validation-rmse:6.72994
[23]	validation-rmse:6.70547
[24]	validation-rmse:6.68390
[25]	validation-rmse:6.66421
[26]	validation-rmse:6.64806
[27]	validation-rmse:6.63280
[28]	validation-rmse:6.61924
[29]	validation-rmse:6.60773
[30]	validation-rmse:6.59777
[31]	validation-rmse:6.58875
[32]	validation-rmse:6.58107
[33]	validation-rmse:6.57217
[34]	validation-rmse:

c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\xgboost\core.py:160: UserWarning: [10:40:00] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1240: Saving into deprecated binary model format, please consider using `json` or `ubj`. Model format will default to JSON in XGBoost 2.2 if not specified.
  warnings.warn(smsg, UserWarning)


In [17]:
from sklearn.ensemble import (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
)


from sklearn.svm import LinearSVR


mlflow.sklearn.autolog(log_datasets=False)


for model_class in (
    RandomForestRegressor,
    GradientBoostingRegressor,
    ExtraTreesRegressor,
    LinearSVR,
):

    with mlflow.start_run():

        mlflow.log_param(
            "train-data-path", "data/green_tripdata_2021-01.parquet"
        )

        mlflow.log_param(
            "valid-data-path", "data/green_tripdata_2021-02.parquet"
        )

        mlflow.log_artifact(
            "models/preprocessor.b", artifact_path="preprocessor"
        )

        mlmodel = model_class()

        mlmodel.fit(X_train, y_train)

        y_pred = mlmodel.predict(X_val)

        rmse = root_mean_squared_error(y_val, y_pred)

        mlflow.log_metric("rmse", rmse)

2024/05/28 21:52:23 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/28 22:06:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils."
c:\Users\ramsi\anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\svm\_base.py:1235: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
